In [1]:
from absbox.local.generic import Generic

In [2]:
test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee",["acc01"],['trusteeFee']]
         ,["payInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None)

In [3]:
from absbox import API
localAPI = API("https://absbox.org/api/latest")

In [75]:
import pandas as pd

loan_tape = pd.read_csv("~/Downloads/U90133_3132H3EE9_COLLAT_CURRENT.txt",sep="|",dtype={'First Payment Date':str})

In [76]:
loan_tape.columns.to_list()

['Loan Identifier',
 'Loan Correction Indicator',
 'Prefix',
 'Security Identifier',
 'CUSIP',
 'Mortgage Loan Amount',
 'Issuance Investor Loan UPB',
 'Current Investor Loan UPB',
 'Amortization Type',
 'Original Interest Rate',
 'Issuance Interest Rate',
 'Current Interest Rate',
 'Issuance Net Interest Rate',
 'Current Net Interest Rate',
 'First Payment Date',
 'Maturity Date',
 'Loan Term',
 'Remaining Months to Maturity',
 'Loan Age',
 'Loan-To-Value (LTV)',
 'Combined Loan-To-Value (CLTV)',
 'Debt-To-Income (DTI)',
 'Borrower Credit Score',
 'Filler',
 'Filler.1',
 'Filler.2',
 'Number of Borrowers',
 'First Time Home Buyer Indicator',
 'Loan Purpose',
 'Occupancy Status',
 'Number of Units',
 'Property Type',
 'Channel',
 'Property State',
 'Seller Name',
 'Servicer Name',
 'Mortgage Insurance Percent',
 'Mortgage Insurance Cancellation Indicator',
 'Government Insured Guarantee',
 'Assumability Indicator',
 'Interest Only Loan Indicator',
 'Interest Only First Principal and In

In [77]:
loan_tape[loan_tape['Days Delinquent']!="Current"]['Days Delinquent']

60     150-179
117     90-119
135      30-59
193      30-59
Name: Days Delinquent, dtype: object

In [78]:
d = loan_tape[['Mortgage Loan Amount','Current Investor Loan UPB','Amortization Type','Original Interest Rate','First Payment Date','Loan Term','Remaining Months to Maturity','Index', 'Origination Mortgage Loan Amount','Origination Interest Rate','Current Interest Rate','Origination Loan Term','Days Delinquent']]

In [83]:
d['First Payment Date']

0      082012
1      082012
2      082012
3      082012
4      082012
        ...  
263    082012
264    082012
265    082012
266    082012
267    092012
Name: First Payment Date, Length: 268, dtype: object

In [85]:
pd.DateOffset(months=1)

<DateOffset: months=1>

In [82]:
pd.to_datetime(d['First Payment Date'])

0     2012-08-20
1     2012-08-20
2     2012-08-20
3     2012-08-20
4     2012-08-20
         ...    
263   2012-08-20
264   2012-08-20
265   2012-08-20
266   2012-08-20
267   2012-09-20
Name: First Payment Date, Length: 268, dtype: datetime64[ns]

In [86]:
pd.to_datetime(d['First Payment Date']) - pd.DateOffset(months=1)

0     2012-07-20
1     2012-07-20
2     2012-07-20
3     2012-07-20
4     2012-07-20
         ...    
263   2012-07-20
264   2012-07-20
265   2012-07-20
266   2012-07-20
267   2012-08-20
Name: First Payment Date, Length: 268, dtype: datetime64[ns]

In [122]:
# use an intermedia dataframe to speed up via vectorize computation 
size_of_loan_tape = loan_tape.shape[0]

mapped_df = pd.DataFrame()
mapped_df['originBalance'] = d['Mortgage Loan Amount']
mapped_df['originRate'] = [["fix",_/100] for _ in d['Original Interest Rate'].to_list() ]
mapped_df['originTerm'] = d['Loan Term']
mapped_df['freq'] = "Monthly" 
mapped_df['type'] = "Level"
mapped_df['originDate'] = (pd.to_datetime(d['First Payment Date']) - pd.DateOffset(months=1)).map(lambda x: x.strftime("%Y-%m-%d"))
mapped_df['currentBalance'] = d['Current Investor Loan UPB']
mapped_df['currentRate'] = d['Current Interest Rate']/100
mapped_df['remainTerm'] = d['Remaining Months to Maturity']
mapped_df['status'] =  d['Days Delinquent'].map(lambda x: "Current" if x=='Current' else "Defaulted")


In [123]:
origin_fields = set(['originBalance', 'originRate', 'originTerm', 'freq', 'type', 'originDate'])
current_fields = set(['currentBalance', 'currentRate', 'remainTerm', 'status'])

mortgages = [ ["Mortgage"
    ,{k:v for k,v in x.items() if k in origin_fields}
    ,{k:v for k,v in x.items() if k in current_fields}]
    for x in  mapped_df.to_dict(orient="records") ]

In [124]:
mortgages

[['Mortgage',
  {'originBalance': 127000.0,
   'originRate': ['fix', 0.045],
   'originTerm': 360,
   'freq': 'Monthly',
   'type': 'Level',
   'originDate': '2012-07-20'},
  {'currentBalance': 98530.54,
   'currentRate': 0.045,
   'remainTerm': 229,
   'status': 'Current'}],
 ['Mortgage',
  {'originBalance': 128000.0,
   'originRate': ['fix', 0.045],
   'originTerm': 360,
   'freq': 'Monthly',
   'type': 'Level',
   'originDate': '2012-07-20'},
  {'currentBalance': 87959.84,
   'currentRate': 0.045,
   'remainTerm': 191,
   'status': 'Current'}],
 ['Mortgage',
  {'originBalance': 135000.0,
   'originRate': ['fix', 0.045],
   'originTerm': 360,
   'freq': 'Monthly',
   'type': 'Level',
   'originDate': '2012-07-20'},
  {'currentBalance': 94164.6,
   'currentRate': 0.045,
   'remainTerm': 194,
   'status': 'Current'}],
 ['Mortgage',
  {'originBalance': 119000.0,
   'originRate': ['fix', 0.04375],
   'originTerm': 360,
   'freq': 'Monthly',
   'type': 'Level',
   'originDate': '2012-07-2

In [129]:
loan_level_deal = Generic(
    "loan_level_deal"
    ,{"cutoff":"2023-03-01","closing":"2023-02-15","firstPay":"2023-04-20"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2042-01-01"}
    ,{'assets':mortgages}  #<<<<<--- here
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":37498392.54
             ,"rate":0.03
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}}),)
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee",["acc01"],['trusteeFee']]
         ,["payInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None)

In [130]:
r = localAPI.run(loan_level_deal ,assumptions=[] ,read=True)
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum
Date,,,,,,,,,
2023-03-31,37725124.62,124231.49,129083.05,0,0,0,0,0.043790,None
2023-04-30,37614262.48,110862.14,142452.40,0,0,0,0,0.043790,None
2023-05-31,37498392.54,115869.94,137444.60,0,0,0,0,0.043789,None
2023-06-30,37386674.31,111718.23,141596.31,0,0,0,0,0.043789,None
2023-07-31,37269972.85,116701.46,136613.08,0,0,0,0,0.043789,None
...,...,...,...,...,...,...,...,...,...
2041-08-31,1898370.55,183353.14,7854.84,0,0,0,0,0.043778,None
2041-09-30,1711315.89,179981.18,7135.62,0,0,0,0,0.043775,None
2041-10-31,1529018.57,178995.91,6235.38,0,0,0,0,0.043774,None
